<a href="https://colab.research.google.com/github/rafaelportomoura/ufla-gcc128-inteligencia-artificial/blob/main/kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho K-means

Aluno: Rafael Porto Vieira de Moura  
Matrícula: 201820274

## Background


### K-Means

O algoritmo K-Means é uma técnica de agrupamento de dados que visa dividir um conjunto de dados em k grupos, onde k é um número pré-definido, com base na similaridade dos seus elementos. O algoritmo funciona em etapas, onde inicialmente são escolhidos k centros de gravidade aleatórios, que serão usados como referência para os demais elementos. Em seguida, cada elemento é associado ao centro de gravidade mais próximo, criando assim os primeiros k grupos. Depois, o centro de gravidade de cada um desses grupos é recalculado e o processo é repetido até que a convergência seja alcançada. O resultado final é um conjunto de k grupos, onde cada grupo representa uma clusterização dos dados, com base nas suas características e na distância entre eles.

A distância utilizada como métrica é a distância euclidiana, representada pela equação abaixo, onde $p$ e $p'$ são dois objetoros representados por vetores no espaço e n é a quantidade de dimensões.


$$ d(p,q) = \sqrt{\sum_{i=1}^{n}{(p - q)^2}} $$

### Data Set Iris

O conjunto de dados Iris é um dos conjuntos mais icônicos e amplamente utilizados em aprendizado de máquina e estatísticas. Ele contém informações sobre três espécies diferentes de plantas de íris: Setosa, Versicolor e Virginica. Para cada uma das 150 amostras de íris, são registradas quatro características: comprimento e largura das sépalas e pétalas. Esse conjunto de dados é frequentemente utilizado para tarefas de classificação e clustering, bem como para ilustrar conceitos fundamentais em análise de dados e visualização. Graças à sua simplicidade e clareza, o conjunto de dados Iris é uma escolha popular para iniciantes em aprendizado de máquina, bem como para pesquisadores que desejam explorar algoritmos e técnicas de análise de dados. O conjunto de dados pode ser acessado no repositório UCI Machine Learning e é uma valiosa ferramenta para estudos e experimentos na área de ciência de dados.

[Página do dataset](https://archive.ics.uci.edu/dataset/53/iris)


## Código



### Cabeçalho

In [20]:
from sklearn.datasets import  load_iris
import numpy as np
import math
from random import uniform

Logger

In [3]:
class Logger:
    def __init__(self,log_type: str):
        self.log_type = log_type if log_type else 'standard'
        self.__error__ = log_type in ['debug','info','standard', 'error']
        self.__info__ = log_type in ['debug','info','standard']
        self.__debug_logs__ = log_type in ['debug','standard']
        self.__log__ = log_type in ['debug','info','standard','log','error']

    def debug(self,*args):
        if(self.__debug_logs__):
            print(*args)
    def info(self,*args):
        if(self.__info__):
            print(*args)
    def error(self,*args):
        if(self.__error__):
            print("❌ ERROR::",*args)
    def log(self,*args):
        if(self.__log__):
            print(*args)

logger = Logger('info')

### K-Means

Distância euclidiana e de Minkowski

In [51]:
def distancia_euclidiana(p: list,q: list) -> float:
    subtraidos = np.subtract(p,q)
    elevado = np.power(subtraidos,2)
    somatorio = np.sum(elevado)
    distancia = math.sqrt(somatorio)
    return distancia

def distancia_minkowski(p: list, q: list, m: int) -> float:
    subtraidos = np.subtract(p,q)
    elevado = np.power(subtraidos, m)
    somatorio = np.sum(elevado)
    distancia = np.power(somatorio, 1/m)
    return distancia

K-Means

In [76]:
class KMeans:
    def __init__(self, numero_de_grupos: int, pontos: list):
        if numero_de_grupos <= 1:
            raise Exception('Número de grupos deve ser maior do que 1!')
        if numero_de_grupos > len(pontos):
            raise Exception('Número de grupos é maior que a quantidade de elementos!')
        self.pontos = pontos
        self.numero_de_grupos = numero_de_grupos

    def agrupa(self, k_centros_iniciais: list) -> list:
        grupos = self.agrupa_por_centros(k_centros_iniciais)
        grupos_antigos = None
        while grupos != grupos_antigos:
            novos_centros = self.atualiza_os_centros(grupos)
            grupos_antigos = grupos.copy()
            grupos = self.agrupa_por_centros(novos_centros)
        return grupos

    def agrupa_por_centros(self, centros: list) -> list:
        grupos = []
        for ponto in self.pontos:
            menor_distancia = float('inf')
            grupo_menor_distancia = 0
            for grupo_centro in range(len(centros)):
                distancia = distancia_euclidiana(ponto, centros[grupo_centro])
                if distancia < menor_distancia:
                    menor_distancia = distancia
                    grupo_menor_distancia = grupo_centro

            grupos.append(grupo_menor_distancia)
        return grupos

    def definir_centro(self, pontos: list) -> float:
        dimensoes = len(pontos[0])
        quantidade_de_pontos = len(pontos)

        centro = []
        for dimensao in range(dimensoes):
            soma = 0
            for p in pontos:
                soma += p[dimensao]

            centro.append(soma / float(quantidade_de_pontos))

        return centro

    def atualiza_os_centros(self, grupos: list) -> list:
        pontos_por_grupo = {}

        centros = []

        for i in range(len(grupos)):
            grupo = grupos[i]
            ponto = self.pontos[i]
            if grupo not in pontos_por_grupo:
                pontos_por_grupo[grupo] = []

            pontos_por_grupo[grupo].append(ponto)

        for pontos in pontos_por_grupo.values():
            centros.append(self.definir_centro(pontos))

        return centros

Funções de definir centro

In [62]:
def centros_iniciais_aleatorios(grupos: int, pontos: list):
    dimensoes = len(pontos[0])
    min = {}
    max = {}
    for dimensao in range(dimensoes):
        min[dimensao] = float('inf')
        max[dimensao] = float('-inf')

    for ponto in pontos:
        for dimensao in range(dimensoes):
            valor = ponto[dimensao]
            minimo = min[dimensao]
            maximo = max[dimensao]

            if valor < minimo:
                min[dimensao] = valor

            if valor > maximo:
                max[dimensao] = valor

    centros = []
    for _ in range(grupos):
        ponto_aleatorio = []
        for dimensao in range(dimensoes):
            minimo = min[dimensao]
            maximo = max[dimensao]
            ponto_aleatorio.append(uniform(minimo, maximo))
        centros.append(ponto_aleatorio)
    return centros



In [115]:
def media_dos_dados_reais() -> list:
    return [[5.0, 3.4, 1.5, 0.2], [5.9, 2.8, 4.4, 1.3], [6.5, 3.0, 5.6, 2.0]]

In [125]:
def aleatorio_com_maior_sucesso() -> list:
    return [[5.72071912152898, 3.6380706517816925, 2.627248060419686, 1.9269155465745458],  [5.572188297522861, 2.192750385539847, 6.652582180638826, 0.9504023867442397],  [5.707959601394808, 2.690916668029683, 4.448261218361639, 1.694869384601041]]

Orquestrador


Rodar k-means


In [117]:
def rodar_k_means(grupos: int, pontos: list, k_centros_iniciais: list):
    try:
        logger.debug(grupos, pontos)
        kmeans = KMeans(grupos, pontos)
        resultado = kmeans.agrupa(k_centros_iniciais)
        return resultado
    except Exception as e:
        logger.error(e.__str__())

def porcentagem_de_acerto(agrupados: list, corretos: list):
    acertos = 0
    erros = 0
    total = len(corretos)
    for agrupado, correto in zip(agrupados,corretos):
        if agrupado == correto:
            acertos += 1
        else:
            erros += 1

    return (acertos/total) * 100

### Dataset Iris


Carregamento do dataset

In [23]:
IRIS_PONTOS, IRIS_CLASSES = load_iris(return_X_y=True)

Rodar K-means com Iris

In [126]:
NUMERO_DE_GRUPOS = 3
# k_centros_iniciais = centros_iniciais_aleatorios(grupos, pontos)
k_centros_iniciais = aleatorio_com_maior_sucesso()
# k_centros_iniciais = media_dos_dados_reais()
resultado = rodar_k_means(NUMERO_DE_GRUPOS,IRIS_PONTOS.tolist(), k_centros_iniciais)
acertos = porcentagem_de_acerto(resultado, IRIS_CLASSES)
logger.info(IRIS_CLASSES.tolist())
logger.info(resultado)
logger.log(f"{acertos:.2f}%")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1]
89.33%
